## 样例介绍 ##

传统定义的Optical Character Recognition（光学字符识别）主要完成文档扫描类的工作。

如今，OCR一般指Scene Text Recognition （场景文字识别），主要面向自然场景。 OCR两阶段方法一般包含两个模型，检测模型负责找出图像或视频中的文字位置，识别模型负责将图像信息转换为文本信息。

此样例中，我们使用的检测模型为CTPN，识别模型则是SVTR。

CTPN模型基于Faster RCNN模型修改而来，而SVTR则基于近几年十分流行的Vision Transformer模型。

## 前期准备


* 基础镜像的样例目录中已包含转换后的om模型以及测试图片，如果直接运行，可跳过此步骤。如果需要重新转换模型，可以参考下面的步骤。
* **建议在Linux服务器或者虚拟机转换该模型。**
* 首先我们可以在[这个链接](https://ascend-repo.obs.cn-east-2.myhuaweicloud.com/Atlas%20200I%20DK%20A2/DevKit/downloads/23.0.RC1/Ascend-devkit_23.0.RC1_downloads.xlsx)的表格中找到本样例的依赖文件，下载我们已经准备好了的air模型文件与onnx模型文件。air是Mindspore框架的模型结构，而ONNX则是开源的离线推理模型框架。
* 为了能进一步优化模型推理性能，我们需要将其转换为om模型进行使用
    * 以下为转换指令：
        CTPN模型:

        ```shell
        atc --model=./cnnctc.air --framework=1 --soc_version=Ascend310B1 --output=ctpn --log=info
        ```
        SVTR模型:
        ```shell
        atc --model=./svtr.onnx --framework=5 --input_shape='x:1,3,48,1440' --input_format=NCHW --soc_version=Ascend310B1 --output=svtr
        ```
    * 其中转换参数的含义为：  
        * --model：输入模型路径
        * --framework：原始网络模型框架类型，1表示air，5表示ONNX
        * --output：输出模型路径
        * --input_format：输入Tensor的内存排列方式
        * --input_shape：指定模型输入数据的shape
        * --log：日志级别
        * --soc_version：昇腾AI处理器型号

## 模型推理实现 ##

下面我们会简单得介绍一下模型推理的几个必须的步骤

### 前处理 ###

前处理的主要目的是将输入处理成模型需要的输入格式，最常见的比如NLP任务中使用的tokenlizer，CV任务中使用的标准化/色域转换/缩放等操作

#### CTPN ####

CTPN模型的前处理由简单的resize和 Normalization组成，代码如下所示


    def preprocess(self, img):
        # resize image and convert dtype to fp32
        dst_img = cv2.resize(img, (int(self.model_width), int(self.model_height))).astype(np.float32)

        # normalization
        dst_img -= self.mean
        dst_img /= self.std

        # hwc to chw
        dst_img = dst_img.transpose((2, 0, 1))

        # chw to nchw
        dst_img = np.expand_dims(dst_img, axis=0)
        dst_img = np.ascontiguousarray(dst_img).astype(np.float32)
        return dst_img


#### SVTR ####

SVTR模型的前处理和CTPN稍有不一样，除了resize和 Normalization之外，为了保证图片缩放后的宽高比以及符合模型输入的shape，我们还需要对其做Padding操作，代码如下所示



    def preprocess(self, img):
        h, w, _ = img.shape
        ratio = w / h
        if math.ceil(ratio * self.model_height) > self.model_width:
            resize_w = self.model_width
        else:
            resize_w = math.ceil(ratio * self.model_height)

        img = cv2.resize(img, (resize_w, self.model_height))

        _, w, _ = img.shape
        padding_w = self.model_width - w
        img = cv2.copyMakeBorder(img, 0, 0, 0, padding_w, cv2.BORDER_CONSTANT, value=0.).astype(np.float32)
        img *= self.scale
        img -= self.mean
        img /= self.std

        # hwc to chw
        dst_img = img.transpose((2, 0, 1))

        # chw to nchw
        dst_img = np.expand_dims(dst_img, axis=0)
        dst_img = np.ascontiguousarray(dst_img).astype(np.float32)

        return dst_img


### 推理接口 ###

两个模型的推理接口部分是相似的，在复制输入数据至模型输入的地址之后我们调用execute api就可以推理了,代码如下图所示


    def infer(self, tensor):
        np_ptr = acl.util.bytes_to_ptr(tensor.tobytes())
        # copy input data from host to device
        ret = acl.rt.memcpy(self.input_data[0]["buffer"], self.input_data[0]["size"], np_ptr,
                            self.input_data[0]["size"], ACL_MEMCPY_HOST_TO_DEVICE)

        # infer exec
        ret = acl.mdl.execute(self.model_id, self.load_input_dataset, self.load_output_dataset)

        inference_result = []

        for i, item in enumerate(self.output_data):
            buffer_host, ret = acl.rt.malloc_host(self.output_data[i]["size"])
            # copy output data from device to host
            ret = acl.rt.memcpy(buffer_host, self.output_data[i]["size"], self.output_data[i]["buffer"],
                                self.output_data[i]["size"], ACL_MEMCPY_DEVICE_TO_HOST)

            data = acl.util.ptr_to_bytes(buffer_host, self.output_data[i]['size'])
            data = np.frombuffer(data, dtype=self.output_dtypes[i]).reshape(self.output_shapes[i])
            inference_result.append(data)

        return inference_result


### 后处理 ###

#### CTPN ####

与传统的检测模型的后处理相似，CTPN的后处理也有着NMS等操作。与之不同的是CTPN使用了文本线构造方法使多个bbox组成了一个文本框。代码如下所示

    def postprocess(self, output):
        proposal = output[0]
        proposal_mask = output[1]
        all_box_tmp = proposal
        all_mask_tmp = np.expand_dims(proposal_mask, axis=1)
        using_boxes_mask = all_box_tmp * all_mask_tmp
        textsegs = using_boxes_mask[:, 0:4].astype(np.float32)
        scores = using_boxes_mask[:, 4].astype(np.float32)
        bboxes = detect(textsegs, scores[:, np.newaxis], (self.model_height, self.model_width))
        return bboxes

#### SVTR ####

与大部分文本识别模型相似，SVTR的后处理就是一个简单的Argmax操作之后查表的过程，代码如下所示

    def postprocess(self, output):
        output = np.argmax(output[0], axis=2).reshape(-1)
        ans = []
        last_char = ''
        for i, char in enumerate(output):
            if char and self.labels[char] != last_char:
                ans.append(self.labels[char])
            last_char = self.labels[char]
        return ''.join(ans)


## 样例运行 ##

In [ ]:
import os
import acl
import cv2
import tqdm
from PIL import Image, ImageDraw

from model import CTPN, SVTR
from utils import get_images_from_path, img_read

import IPython

def init_acl(device_id=0):
    acl.init()
    ret = acl.rt.set_device(device_id)
    if ret:
        raise RuntimeError(ret)
    context, ret = acl.rt.create_context(device_id)
    if ret:
        raise RuntimeError(ret)
    print('Init ACL Successfully')
    return context


def deinit_acl(context, device_id=0):
    ret = acl.rt.destroy_context(context)
    if ret:
        raise RuntimeError(ret)

    ret = acl.rt.reset_device(device_id)
    if ret:
        raise RuntimeError(ret)
    print('ACL resources were released successfully.')
    


def get_images_from_path(img_path):
    img_list = []
    if os.path.isfile(img_path):
        img_list.append(img_path)
    if os.path.isdir(img_path):
        for file in os.listdir(img_path):
            img_list.append(os.path.join(img_path, file))
    return img_list


def img_read(path):
    path = os.path.realpath(path)
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    if img is None:
        raise ValueError(f"cannot decode file:{path}")
    return img

In [ ]:
CTPN_MODEL_PATH = './ctpn.om'
SVTR_MODEL_PATH = './svtr.om'
SVTR_DICT_PATH = './ppocr_keys_v1.txt'
DEVICE_ID = 0

In [ ]:
context = init_acl(DEVICE_ID)

In [ ]:
det_model = CTPN(model_path=CTPN_MODEL_PATH, device_id=DEVICE_ID)

In [ ]:
rec_model = SVTR(model_path=SVTR_MODEL_PATH, dict_path=SVTR_DICT_PATH,
                     device_id=DEVICE_ID)

In [ ]:
IMAGE_PATH = './sample.png'

In [ ]:
if not os.path.exists('infer_result'):
    os.makedirs('infer_result')
ans = {}

img_src = img_read(IMAGE_PATH)
basename = os.path.basename(IMAGE_PATH)
print(f'start infer image: {basename}')
name, ext = os.path.splitext(basename)
image_h, image_w = img_src.shape[:2]

det_input_tensor = det_model.preprocess(img_src)

output = det_model.infer(det_input_tensor)

bboxes = det_model.postprocess(output)
im = Image.open(IMAGE_PATH)
draw = ImageDraw.Draw(im)
ans['image_name'] = basename
ans['result'] = []
for bbox in bboxes:
    bbox_detail = {}
    x1 = int(bbox[0] / det_model.model_width * image_w)
    y1 = int(bbox[1] / det_model.model_height * image_h)
    x2 = int(bbox[2] / det_model.model_width * image_w)
    y2 = int(bbox[3] / det_model.model_height * image_h)
    draw.line([(x1, y1), (x1, y2), (x2, y2), (x2, y1), (x1, y1)], fill='red', width=2)
    bbox_detail['bbox'] = [x1, y1, x1, y2, x2, y2, x2, y1]
    res  = ','.join(map(str,bbox_detail['bbox']))
    print(f'det result: {res}')
    crop_img = img_src[y1:y2, x1:x2]

    rec_input_tensor = rec_model.preprocess(crop_img)
    rec_output = rec_model.infer(rec_input_tensor)
    bbox_detail['text'] = rec_model.postprocess(rec_output)
    print(f'rec result: {bbox_detail["text"]}')
    ans['result'].append(bbox_detail)

im.save(os.path.join('infer_result', name + '_res' + ext))
    

In [ ]:
IPython.display.Image(filename=os.path.join('infer_result', name + '_res' + ext)) 

In [ ]:
det_model.deinit()
rec_model.deinit()
deinit_acl(context, DEVICE_ID)

## 总结与扩展 ##

以上就是OCR样例的全部内容了，这个流程可以适用于所有两阶段的OCR任务。我们也可以将检测模型换成DBNet或者DBNet++等文本检测模型。或者将识别模型替换为CRNN等文本识别模型。需要注意的是，整个流程最重要的部分是模型前后处理部分的代码，比如Resize的宽高参数，在OCR任务中，前后处理的参数将会对识别结果产生巨大的影响。